In [1]:
import requests as rq
import requests_cache
import lxml.html as lx
from bs4 import BeautifulSoup
import pandas as pd
import re
from io import StringIO

In [2]:
urls = ["https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-8/table-8-state-cuts/california.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2018/crime-in-the-u.s.-2018/tables/table-8/table-8-state-cuts/california.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2017/crime-in-the-u.s.-2017/tables/table-8/table-8-state-cuts/california.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-6/table-6-state-cuts/california.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2015/crime-in-the-u.s.-2015/tables/table-8/table-8-state-pieces/table_8_offenses_known_to_law_enforcement_california_by_city_2015.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2014/crime-in-the-u.s.-2014/tables/table-8/table-8-by-state/Table_8_Offenses_Known_to_Law_Enforcement_by_California_by_City_2014.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_california_by_city_2013.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2012/crime-in-the-u.s.-2012/tables/8tabledatadecpdf/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_by_california_by_city_2012.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2011/crime-in-the-u.s.-2011/tables/table8statecuts/table_8_offenses_known_to_law_enforcement_california_by_city_2011.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2010/crime-in-the-u.s.-2010/tables/table-8/10tbl08ca.xls"]
session = requests_cache.CachedSession("crime_data")

In [3]:
def get_data(url_lst):
    all_data = pd.DataFrame()
    year = 2019
    names_lst = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
    for url in url_lst:
        request = session.get(url)
        soup = BeautifulSoup(request.content, 'html.parser')
        table = soup.find("table", {"class": "data"})
        data_frame = pd.read_html(StringIO(str(table)))[0]
        if year >= 2014 and year <= 2016:
            data_frame = data_frame.drop(data_frame.columns[5], axis = 1)
        elif year == 2013:
            data_frame = data_frame.drop(data_frame.columns[4], axis = 1)
        data_frame.columns = data_frame.columns[:0].to_list() + names_lst
        state_name = "California"
        data_frame["state"] = state_name
        data_frame["year"] = year
        year -= 1
        all_data = pd.concat([all_data, data_frame])
    all_data = all_data.reset_index()
    all_data = all_data.drop(["index"], axis= 1)
    return all_data

In [4]:
california_data = get_data(urls)

In [5]:
california_data

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson,state,year
0,Adelanto,34491,276,1,20.0,42,213,459.0,136.0,209.0,114,14,California,2019
1,Agoura Hills,20490,21,0,6.0,4,11,306.0,66.0,223.0,17,0,California,2019
2,Alameda,78907,162,0,7.0,94,61,2579.0,218.0,1958.0,403,29,California,2019
3,Albany,20083,40,0,8.0,21,11,685.0,105.0,534.0,46,1,California,2019
4,Alhambra,84837,161,2,11.0,89,59,1749.0,259.0,1303.0,187,8,California,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4601,Yountville,3293,5,0,1.0,1,3,57.0,9.0,47.0,1,1,California,2010
4602,Yreka,7345,54,0,2.0,2,50,263.0,69.0,176.0,18,2,California,2010
4603,Yuba City,62694,216,2,22.0,51,141,1783.0,409.0,1222.0,152,11,California,2010
4604,Yucaipa,50939,139,0,10.0,31,98,805.0,233.0,464.0,108,9,California,2010


In [6]:
crime_2009 = pd.read_excel("../raw_data/09tbl08ca.xls")
crime_2009.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2009 = crime_2009.tail(crime_2009.shape[0]-4)
crime_2009 = crime_2009.head(crime_2009.shape[0]-1)
cols = ['population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
        'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2009[cols] = crime_2009[cols].apply(pd.to_numeric, errors = "coerce", axis = 1)
crime_2009["state"] = "California"
crime_2009["year"] = 2009
crime_2009

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson,state,year
4,Adelanto,30045.0,272.0,2.0,8.0,40.0,222.0,797.0,346.0,324.0,127.0,13.0,California,2009
5,Agoura Hills,22469.0,27.0,0.0,2.0,6.0,19.0,308.0,105.0,192.0,11.0,6.0,California,2009
6,Alameda,70372.0,199.0,4.0,13.0,87.0,95.0,1955.0,325.0,1390.0,240.0,12.0,California,2009
7,Albany,15950.0,45.0,0.0,1.0,36.0,8.0,600.0,112.0,381.0,107.0,17.0,California,2009
8,Alhambra,85956.0,248.0,0.0,15.0,133.0,100.0,1874.0,360.0,1209.0,305.0,4.0,California,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,Yountville,3265.0,4.0,0.0,2.0,0.0,2.0,43.0,11.0,31.0,1.0,0.0,California,2009
460,Yreka,7379.0,55.0,0.0,2.0,4.0,49.0,224.0,48.0,162.0,14.0,0.0,California,2009
461,Yuba City,62495.0,196.0,1.0,16.0,44.0,135.0,1555.0,289.0,1130.0,136.0,10.0,California,2009
462,Yucaipa,50782.0,128.0,4.0,13.0,19.0,92.0,1030.0,247.0,640.0,143.0,22.0,California,2009


In [7]:
#older_rates = excel_df(file_paths)
crime_2008 = pd.read_excel('../raw_data/08tbl08ca.xls')
crime_2008 = crime_2008.drop(["Unnamed: 12", "Unnamed: 13", "Unnamed: 14"], axis = 1)
crime_2008.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2008 = crime_2008.tail(crime_2008.shape[0]-4)
crime_2008 = crime_2008.head(crime_2008.shape[0]-1)
crime_2008 = crime_2008.dropna(axis = 0, how = 'all')
crime_2008 = crime_2008.head(crime_2008.shape[0] - 3)
cols = ['population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
        'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2008[cols] = crime_2008[cols].apply(pd.to_numeric, errors = "coerce", axis = 1)
crime_2008["state"] = "California"
crime_2008["year"] = 2008
crime_2008

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson,state,year
4,Adelanto,29214.0,185.0,2.0,9.0,30.0,144.0,822.0,348.0,361.0,113.0,12.0,California,2008
5,Agoura Hills,22619.0,43.0,0.0,2.0,9.0,32.0,376.0,96.0,257.0,23.0,0.0,California,2008
6,Alameda,69998.0,210.0,2.0,9.0,107.0,92.0,1875.0,325.0,1278.0,272.0,9.0,California,2008
7,Albany,15909.0,50.0,0.0,3.0,30.0,17.0,650.0,88.0,435.0,127.0,0.0,California,2008
8,Alhambra,86404.0,276.0,1.0,8.0,156.0,111.0,2082.0,447.0,1270.0,365.0,9.0,California,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Yountville,3272.0,1.0,0.0,1.0,0.0,0.0,45.0,14.0,26.0,5.0,0.0,California,2008
459,Yreka,7358.0,44.0,0.0,5.0,4.0,35.0,222.0,49.0,161.0,12.0,3.0,California,2008
460,Yuba City,62595.0,231.0,4.0,14.0,65.0,148.0,1999.0,383.0,1384.0,232.0,13.0,California,2008
461,Yucaipa,51202.0,83.0,1.0,9.0,38.0,35.0,938.0,252.0,549.0,137.0,20.0,California,2008


In [8]:
crime_2007 = pd.read_excel("../raw_data/07tbl08ca.xls")
crime_2007 = crime_2007.drop(["Table 8"], axis = 1)
crime_2007.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2007 = crime_2007.tail(crime_2007.shape[0]-3)
crime_2007 = crime_2007.head(crime_2007.shape[0]-3)
cols = ['population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
        'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2007[cols] = crime_2007[cols].apply(pd.to_numeric, errors = "coerce", axis = 1)
crime_2007["state"] = "California"
crime_2007["year"] = 2007
crime_2007

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson,state,year
3,Adelanto,28719.0,153.0,5.0,7.0,33.0,108.0,758.0,354.0,308.0,96.0,11.0,California,2007
4,Agoura Hills,22966.0,49.0,0.0,3.0,9.0,37.0,348.0,76.0,254.0,18.0,6.0,California,2007
5,Alameda,70445.0,205.0,2.0,9.0,106.0,88.0,1976.0,304.0,1427.0,245.0,17.0,California,2007
6,Albany,15889.0,45.0,0.0,3.0,33.0,9.0,776.0,173.0,474.0,129.0,3.0,California,2007
7,Alhambra,87729.0,285.0,1.0,8.0,192.0,84.0,2120.0,426.0,1301.0,393.0,16.0,California,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Yountville,3312.0,5.0,0.0,1.0,0.0,4.0,58.0,11.0,46.0,1.0,0.0,California,2007
457,Yreka,7536.0,54.0,1.0,5.0,4.0,44.0,297.0,65.0,218.0,14.0,2.0,California,2007
458,Yuba City,61881.0,245.0,2.0,24.0,54.0,165.0,1913.0,322.0,1361.0,230.0,10.0,California,2007
459,Yucaipa,51624.0,52.0,2.0,9.0,16.0,25.0,914.0,250.0,504.0,160.0,3.0,California,2007


In [9]:
crime_2006 = pd.read_excel("../raw_data/06tbl08ca.xls")
crime_2006 = crime_2006.drop(["Unnamed: 12"], axis=1)
crime_2006.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2006 = crime_2006.dropna(axis = 0, how = 'all')
crime_2006 = crime_2006.tail(crime_2006.shape[0]-4)
crime_2006 = crime_2006.head(crime_2006.shape[0]-2)
cols = ['population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
        'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2006[cols] = crime_2006[cols].apply(pd.to_numeric, errors = "coerce", axis = 1)
crime_2006["state"] = "California"
crime_2006["year"] = 2006
crime_2006

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson,state,year
4,Adelanto,24579,136,2,14,35,85,692,238,305,149,7,California,2006
5,Agoura Hills,22970,41,0,3,7,31,338,86,229,23,3,California,2006
6,Alameda,71212,221,2,7,81,131,1912,323,1297,292,9,California,2006
7,Albany,16138,54,0,4,40,10,796,134,539,123,5,California,2006
8,Alhambra,88197,297,1,19,158,119,2472,443,1572,457,14,California,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Yountville,3337,2,0,0,0,2,40,6,32,2,0,California,2006
457,Yreka,7361,53,0,0,3,50,235,32,189,14,3,California,2006
458,Yuba City,59156,250,2,21,50,177,2029,459,1326,244,22,California,2006
459,Yucaipa,49542,92,4,8,15,65,785,185,467,133,6,California,2006


In [10]:
cali_lst = ["../raw_data/cali_2005.xlsx","../raw_data/cali_2004.xlsx","../raw_data/cali_2003.xlsx"]

In [11]:
def get_cali(file_lst):
    all_data = pd.DataFrame()
    year = 2005
    for file in file_lst:
        df = pd.read_excel(file)
        df.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
        df["state"] = "California"
        df["year"] = year
        year -= 1
        all_data = pd.concat([all_data, df],)
    return all_data

In [12]:
final_years = get_cali(cali_lst)

In [13]:
final_years.dtypes

city                                    object
population                               int64
violent crime                            int64
murder and nonnegligent manslaughter     int64
rape                                     int64
robbery                                  int64
aggravated assault                       int64
property crime                           int64
burglary                                 int64
larceny-theft                            int64
motor vehicle theft                      int64
arson                                    int64
state                                   object
year                                     int64
dtype: object

In [14]:
california_data = pd.concat([california_data, crime_2009, crime_2008, crime_2007, crime_2006, final_years])

In [21]:
crime_2020 = pd.read_excel("../raw_data/cali_2020.xlsx")
crime_2020 = crime_2020.tail(crime_2020.shape[0] - 3)
crime_2020.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
cols = ['population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
        'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2020[cols] = crime_2020[cols].apply(pd.to_numeric, errors = "coerce", axis = 1)
crime_2020["state"] = "California"
crime_2020["year"] = 2020
crime_2020.dtypes

city                                    object
population                               int64
violent crime                            int64
murder and nonnegligent manslaughter     int64
rape                                     int64
robbery                                  int64
aggravated assault                       int64
property crime                           int64
burglary                                 int64
larceny-theft                            int64
motor vehicle theft                      int64
arson                                    int64
state                                   object
year                                     int64
dtype: object

In [16]:
crime_2021 = pd.read_excel("../raw_data/cali_2021.xlsx")
crime_2021.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2021["state"] = "California"
crime_2021["year"] = 2021
crime_2021

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson,state,year
0,Carlsbad,116633,235,2,26,40,167,2032,280,1544,208,18,California,2021
1,Chula Vista,275978,871,6,41,249,575,3334,400,1989,945,50,California,2021
2,El Cajon,102665,525,2,43,113,367,1860,261,1179,420,26,California,2021
3,Escondido,150507,545,3,27,140,375,2644,333,1741,570,30,California,2021
4,La Mesa,59968,157,2,12,42,101,1145,181,751,213,15,California,2021
5,National City,61171,402,4,19,91,288,1291,132,831,328,16,California,2021
6,Oceanside,175335,771,6,83,163,519,3346,426,2433,487,40,California,2021
7,San Diego,1434673,5404,63,444,1040,3857,26348,3286,16601,6461,150,California,2021


In [17]:
crime_2022 = pd.read_excel("../raw_data/cali_2022.xlsx")
crime_2022.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2022["state"] = "California"
crime_2022["year"] = 2022
crime_2022

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson,state,year
0,Adelanto,38380,344,0,11,37,296,359,71,193,95,NaN,California,2022
1,Agoura Hills,19397,32,0,5,4,23,255,69,163,23,NaN,California,2022
2,Alameda,74441,232,1,17,100,114,3200,297,2302,601,20.0,California,2022
3,Albany,19011,40,0,5,16,19,564,70,432,62,12.0,California,2022
4,Alhambra,79703,160,0,12,73,75,2044,251,1453,340,6.0,California,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,Yountville,3349,16,0,2,1,13,38,2,33,3,NaN,California,2022
437,Yreka,7865,118,0,5,0,113,370,35,278,57,NaN,California,2022
438,Yuba City,69122,380,2,46,63,269,1565,187,1135,243,9.0,California,2022
439,Yucaipa,54873,190,1,13,21,155,714,154,447,113,NaN,California,2022


In [18]:
california_data = pd.concat([california_data, crime_2020, crime_2021, crime_2022])

In [19]:
california_data = california_data.sort_values(by = ["year", "city"], ascending=[False, True])
california_data = california_data.reset_index()
california_data = california_data.drop(["index"], axis=1)

In [20]:
california_data.to_csv("../clean_data/cali_crime.csv")